In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Input
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load the movie dataset
data = pd.read_csv('transformed_movie_data6.csv')

# Assuming the dataset has a 'release_date' and a 'revenue' column
# Convert 'release_date' column to datetime if it's not already in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Set 'release_date' column as the index
data.set_index('Date', inplace=True)

# Select 'revenue' column as the target variable (y)
y = data['Opening']

# Select features (X), you may choose more relevant features
X = data.drop('Opening', axis=1)

In [3]:
endog = data['Opening']

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
endog_scaled = scaler.fit_transform(np.array(endog).reshape(-1, 1))

In [5]:
def create_lstm_dataset(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Define the time step for LSTM
time_step = 100  # You may adjust this

In [6]:
X_lstm, y_lstm = create_lstm_dataset(endog_scaled, time_step)

In [7]:
split = int(0.8 * len(X_lstm))
X_train_lstm, X_test_lstm = X_lstm[:split], X_lstm[split:]
y_train_lstm, y_test_lstm = y_lstm[:split], y_lstm[split:]

In [8]:
X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], 1))
X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], X_test_lstm.shape[1], 1))

In [9]:
# Build LSTM model
model_lstm = Sequential()
model_lstm.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model_lstm.add(LSTM(units=50, return_sequences=True))
model_lstm.add(LSTM(units=50, return_sequences=True))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(units=1))

In [10]:
# Compile the model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.1541
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0956
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0864
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0843
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0865
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0872
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0866
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0883
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0939
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0878
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0881
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0862
Epoch 13/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0859
Epoch 14/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0869
Epoch 15/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - lo

In [12]:
y_pred_lstm = model_lstm.predict(X_test_lstm)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step


In [13]:
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)

In [19]:
def mean_squared_error(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [20]:
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)
rmse_lstm = np.sqrt(mse_lstm)
print("RMSE (LSTM):", rmse_lstm)

RMSE (LSTM): 0.3016819106687402
